In [5]:
import boto
from boto.s3.key import Key
import requests
key = 'enter s3 key'
secret = 'enter s3 secret'
import s3fs
import numpy as np
import pandas as pd
import random
import string
import datetime
import random


def generate_boolean():
    return bool(random.getrandbits(1))

def generate_number(minx=0, maxx=100, is_int=False):
    return random.randint(minx, maxx) if is_int else random.uniform(minx, maxx)

def generate_string_data(length):
    return ''.join(random.sample(string.ascii_letters, length))

def generate_hash(length):
    return ''.join(random.sample(string.ascii_letters + string.digits, length))

def generate_list_hash(counts, length):
    return [
        generate_hash(length)
        for _ in range(counts)
    ]


def generate_array_data_type(item_type, n_items=1, minx=0, maxx=100, length=10, item_predefined_values=[],
                            k = 9, theta = 0.5,mu = 0,sigma = 0):
    if item_type == "string":
        return [generate_string_data(length) for _ in range(n_items)]
    elif item_type == "integer":
        return [generate_number(minx, maxx, is_int=True) for _ in range(n_items)]
    elif item_type in {"float", "number"}:
        return [generate_number(minx, maxx, is_int=False) for _ in range(n_items)]
    elif item_type == "boolean":
        return [generate_boolean() for _ in range(n_items)]
    elif item_type == "enum":
        return [random.choice(item_predefined_values) for _ in range(n_items)]
    elif item_type == "gamma":
        return [round(np.random.gamma(k,theta)) for _ in range(n_items)]
    elif item_type == "normal":
        return [round(np.random.normal(mu,sigma)) for _ in range(n_items)]
    else:
        raise UserError("{} is not supported".format(item_type))
        
def generate_dates(start_date, end_date, size):
    """
    Generate random dates within range between start and end.
    Adapted from: https://stackoverflow.com/a/50668285
    """
    # Unix timestamp is in nanoseconds by default, so divide it by
    # 24*60*60*10**9 to convert to days.
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    divide_by = 24 * 60 * 60 * 10 ** 9
    start_u = start.value // divide_by
    end_u = end.value // divide_by
    return pd.to_datetime(np.random.randint(start_u, end_u, size), unit="D")
        
def generate_predefined_list(size, item_predefined_values, p=None):
    """Generate n-length ndarray of genders."""
    if not p:
        # default probabilities
        p = (0.33, 0.33, 0.34)
    return np.random.choice(item_predefined_values, size=size, p=p)



In [1]:
# the list of items to select from transactions
random.seed(42)
# read a list of customer ids

df = pd.read_csv('C:/Users/mm13690/Documents/codes/p_ai/input/premium_customer_ids.csv')
premium_customer_ids = df.customer_ids.tolist()

# generate a list of item ids
item_list = [i for i in range(1,53433)]

##for designer brand seekers
# item_list = [i for i in range(35184,39499)]
# random_item_list = [i for i in range(1,10000)]
# random_item_list = np.random.choice(random_item_list,size = 1500)
# item_list.extend(random_item_list)

hello world


In [ ]:

random.seed(42)

premium_customer_ids_chunks = np.array_split(premium_customer_ids,50)


In [ ]:


# create data for designer brand seekers
designer_brand_seekers = premium_customer_ids_chunks[0]
# loyalists_customer_id = _list
transaction_file = 'dbs_transaction'
campaign_file = 'dbs_campaign'

obj = synthetic_data_simulation(customer_id=designer_brand_seekers,size = len(designer_brand_seekers),transaction_file = transaction_file,
                campaign_file = campaign_file,item_list= item_list,gender = ['Male','Female','Unknown'],
                gender_prob = [0.18,0.71,0.11],
                age = ['< 15', '16 - 19', '20 - 29', '30 - 39', '40 - 49', '50 - 59', '60+','Unknown'],
                age_prob = [0.01, 0.03, 0.24, 0.31, 0.14, 0.08, 0.02, 0.17],
                loyalty_member_flag = [0,1],
                loyalty_member_flag_prob = [0.12,0.88],
                loyalty_member_program = ["Member","Insider","Influencer", "Ambassador"],
                loyalty_member_program_prob = [0.16, 0.24, 0.26, 0.34],
                income_group = ["Less than $49,999", "$50,000 - $74,999",
                "$75,000 - $99,999","$100,000 - $149,999","$150,000 - $199,999", "$200,000+", "Unknown"], 
                income_group_prob = [0, 0.0, 0.05, 0.10, 0.15, 0.60, 0.10],
                store_visit_k = 20,store_visit_theta = 0.5,
                review_k = 6, review_theta = 0.5,
                client_segment = "Premium",
                attempted_segment_split =  "Designer Brand Seekers", 
                signup_date_min = '2015-12-01',signup_date_max = '2017-12-31',

                tx_k = 10, tx_theta = 0.5,order_value_m = 1186,order_value_sd =1186/5,
                scale_2018_factor = 1.04,scale_2020_factor = 0.95,
                
                n_campaign_cust = 15000,campaign ='Valentines Day Gemstone Jewelry Collection',
                offer_id = [1,2,3,4,5,6,7,8],offer_name =  ['3 for 2 Accessories', 'Relaunch Offer – 25% Off',
                '3 for $ 250 Chinos','Free gift on your Birthday','Earn more points','Designer Suits $199.99',
                'Dress shirt and ties extra 20% off','First to shop select brands'],
                offer_prob = [0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125],offer_reward = [50,50,50,50,50,50,50,50],
                top_2_offer = [3,6],
                uplift_feature = ['loyalty_group','age_group'],
                uplift_feature_value = ["Ambassador",'60+'],
                uplift_value = [.20,.15],base_uplift = 0.04,
                campaign_launch_date = '2019-02-01')

df = obj.generate_customer_data()
customer_transaction = obj.generate_transaction_data(df)
customer_transaction, campaign_customer = obj.generate_campaign_data(df,customer_transaction)
obj.write_to_s3(customer_transaction, campaign_customer = campaign_customer)
    

